In [9]:
from functions_aa import *
from ch7.weather.model.model import TcnClassifier
import pickle
model                 = torch.load('data/best_model.pt')



with open('data/learning_data.pickle', 'rb') as handle:
    learning_data = pickle.load(handle)  
    
    
all_data = learning_data['all_data']
    
    

In [13]:
def load_data():    

    raw_df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQwC6jRtVUk-2dkk2W3BDJZTOdsS427LN8Ixo-rQF4Afs6ice0rof7qh_EbnAy5lYEGqX-TCSvjpPyr/pub?gid=1713335339&single=true&output=csv',
                        index_col=['codigo','Tiempo']).drop(['Fecha','Exposicion'], axis=1)
    raw_df.sort_index(inplace=True)
    #adherence_mask = np.where((raw_df['SA'] < 0.5) & (raw_df['Tirosina'] < 600) & (raw_df['NTBC_DBS'] > 15), True, False)
    timepoints_per_patient = pd.Series(raw_df.index.get_level_values(0).values).value_counts()
    timeseries_length      =  timepoints_per_patient.min()
    #raw_df['alfafeto_bad'] = np.where((raw_df['Alfa-fetoprot'] > 10), 'Yes', 'No')
    
    var_names = raw_df.columns.to_list()
    imputed_train_df =  abs(pd.read_csv('imputed_train_df.csv', index_col=['codigo']).drop('Unnamed: 0', axis=1))#.sort_index(inplace=False))
    imputed_val_df   =   abs(pd.read_csv('imputed_val_df.csv', index_col=['codigo']).drop('Unnamed: 0', axis=1))#.sort_index(inplace=False))
    imputed_df       = pd.concat([imputed_val_df, imputed_train_df])#.sort_index(inplace=False)
    IDs              =  imputed_df.index.get_level_values(0).unique()

    #muliindex = itertools.product(IDs, range(1, timeseries_length+1))
    imputed_df.index = pd.MultiIndex.from_product([IDs, range(1, timeseries_length+1)], names=["codigo", "time"])
    imputed_df#.sort_index(inplace=True)
    df = imputed_df
    df.columns = var_names
    df['alfafeto_bad'] = np.where((df['Alfa-fetoprot'] > 10), 'Yes', 'No')

    #print(df['alfafeto_bad'].value_counts())
    df_sin_index = df.reset_index()
    #df_sin_index.drop( 'Alfa-fetoprot', axis=1, inplace=True)
    
    assert 'Alfa-fetoprot' in df_sin_index.columns.to_list()
    
    

    return df_sin_index

df = load_data()

In [10]:
# Explicabilidad con SHAP
import shap

import warnings
warnings.filterwarnings("ignore")

torch.set_grad_enabled(True)

# ESTA ES LA PARTE DE SHAP QUE SE USA PARA LAS COSAS
# explainer = shap.DeepExplainer(model, all_data)
explainer = shap.GradientExplainer(model, all_data)

In [11]:
shap_values = explainer.shap_values(all_data) # AQUI LE PASAS LA DATA SEGMENTADA! 

# Sanity checks de la forma de las estructuras de datos
for shap_explanation in shap_values:
    print(f"{shap_explanation.shape = }")

print(f"{all_data.shape = }")

shap_explanation.shape = (114, 52, 3)
shap_explanation.shape = (114, 52, 3)
all_data.shape = torch.Size([114, 52, 3])


In [14]:
var_names = [s for s in df.columns.to_list() if s not in ['codigo', 'time']]

assert all_data.shape[1] == len(var_names)

In [15]:
# INFO: Los SHAP values estan en un tensor de dimensiones
#   0 - Muestras (pacientes)
#   1 - Features (mediciones)
#   2 - Ventanas (convolucion)
# Para la interpretabilidad de SHAP, y de la visualizacion, es necesario colapsar
# estas dimensiones, usando <numpy_narray>.sum(axis=2), eg. 

# Empaca los valores del tensor en un DF
df2 = pd.DataFrame(
    all_data.detach().numpy().mean(axis=2),
    columns=var_names
)

# Empaca los SHAP_values en un DF
pd_shap_values_alterado_colapsado = pd.DataFrame(
    shap_values[0].sum(axis=2), # 0 - alterado, 1 - normal
    columns=var_names
)

# Computa un indice de columnas por pesos SHAP
reindex_series = (
    pd_shap_values_alterado_colapsado
    .abs()
    .sum(axis='rows')
    .sort_values(ascending=False)
    .index
)

# Reindexa los frames
pd_shap_values_alterado_colapsado = pd_shap_values_alterado_colapsado.reindex(columns=reindex_series)
df2 = df2.reindex(columns=reindex_series)

In [17]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

N_FEATURES : int = 10 # Features a mostrar

for PACIENTE in range(0,16):
    
    print(f"Mostrando ventana de [{(PACIENTE)*6}:{(PACIENTE+1)*6-1}]")

    fig = go.Figure()
    for n, feature in enumerate(df2.columns[:N_FEATURES]):

        fig.add_trace(go.Violin(
            y=pd_shap_values_alterado_colapsado[feature], 
            x=[n]*df2.shape[0], 
            name=feature,
            box_visible=False,
            meanline_visible=False,
            opacity=0.4,
            line_color='black',
            fillcolor='lightgray',
        ))

        # PUNTOS DE LA FIGURA
        fig.add_trace(go.Scatter(
            y=pd_shap_values_alterado_colapsado[feature][(PACIENTE)*6:(PACIENTE+1)*6-1], 
            x=np.random.normal(n,0.1,df2.shape[0])[(PACIENTE)*6:(PACIENTE+1)*6-1], # Dummy con distribucion normal para separar puntos
            mode='markers',
            marker=dict(
                size=15, 
                color=df2[feature].rank(pct=True), 
                colorscale="bluered", 
                opacity=0.8,
            ),
            name=feature,
            orientation='h', # 
        ))

    # Define los Ticks con los nombres de las features
    fig.update_xaxes(
        tickvals=list(range(N_FEATURES)), 
        ticktext=df2[:N_FEATURES].columns,
        tickangle=-45, 
        tickfont=dict(size=20),
    )
    
    # Define la escala del eje vertical, dentro de los SHAP values normales
    fig.update_yaxes(
        title="SHAP Value",
        range=[-3,7],
        tickfont=dict(size=30),
    )

    # Trace vacio para poner la escala de color...
    fig.add_trace(go.Scatter(
        x=[None], y=[None], mode='markers',
        marker=dict(
            colorscale="bluered", 
            showscale=True,
            cmin=-5, cmax=5,
            colorbar=dict(
                thickness=25, 
                tickvals=[-5, 5], 
                ticktext=['Bajo', 'Alto'], 
                outlinewidth=1, 
                tickfont=dict(size=20),
                #orientation='h', # Barra horizontal
            )
        ),
        hoverinfo='none'
    ))
    # ...y le pone una anotacion titulo a dicha barra de color
    # (esto es para la barra vertical al costado derecho)
    fig.add_annotation(text="Valor relativo en la medicion", # TODO: mejor descripcion?
        xref="paper", yref="paper",
        x=1.1, y=0.5, textangle=90, 
        showarrow=False,
        font=dict(
            color="black",
            size=30
        ),
    )

    # Inlcuye una linea para aportes positivos de negativos
    fig.add_hline(
        y=0.0, 
        line_width=2, 
        line_dash="dash", 
        line_color="black"
    )
    fig.update_yaxes(title_font=dict(size=30))


    fig.update_layout(
        # title="Model feature importance",
        template='simple_white',
        showlegend=False, 
        coloraxis_showscale=True,
        autosize=False,
        width=1900, # MODIFICAR PARA ESCALA DE FIGURA
        height=800, # MODIFICAR PARA ESCALA DE FIGURA
    )

    fig.write_image(f"fig/shap_paciente_{PACIENTE}.png")
fig.show()

Mostrando ventana de [0:5]
Mostrando ventana de [6:11]
Mostrando ventana de [12:17]
Mostrando ventana de [18:23]
Mostrando ventana de [24:29]
Mostrando ventana de [30:35]
Mostrando ventana de [36:41]
Mostrando ventana de [42:47]
Mostrando ventana de [48:53]
Mostrando ventana de [54:59]
Mostrando ventana de [60:65]
Mostrando ventana de [66:71]
Mostrando ventana de [72:77]
Mostrando ventana de [78:83]
Mostrando ventana de [84:89]
Mostrando ventana de [90:95]
